In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

from pandas_datareader import data as web 
from datetime import timedelta
from utils import getDateFormat, getDayPortfolio

pd.options.mode.chained_assignment = None 

In [2]:

# Get the data
df_basic = pd.read_csv('./inputs/basic.csv')
df_other = pd.read_csv('./inputs/other.csv')

# Format date fields
df_basic['date'] = pd.to_datetime(df_basic['date'])
df_other['date'] = pd.to_datetime(df_other['date'])

# Split buys and sells into separate dataframes
df_buys = df_basic.loc[df_basic["type"] == "BUY"]
df_sells = df_basic.loc[df_basic["type"] == "SELL"]

# Get the earliest transaction date
minDate = min(df_basic["date"].min(), df_other["date"].min())
yesterday = pd.to_datetime("today") - timedelta(days=1)

# A list of all the tickers
tickers_all = df_basic['ticker'].unique()

# A list of all tickers with their currency
tickers_currency = df_buys[["ticker", "currency"]].drop_duplicates().reset_index(drop=True)

# A list of currently held tickers
tickers_current = getDayPortfolio(df_buys, df_sells)["ticker"].explode().unique() 

In [3]:
# Fill the df with all days from first investment to yeasterday, and linearly interpolate missing values in the given columns
def fillDays(df, columns, start = minDate, end = yesterday):
    # Fill df with all days
    dates = pd.date_range(start, end).to_series(name="date")
    df = pd.merge(dates, df, on="date", how="left", sort=True)

    # Interpolate missing values
    df[columns] = df[columns].interpolate(limit_area = "inside")

    return df.fillna(0)

In [4]:
########################
#  Get Overall Market  #
########################

# Get the data
df_spy = web.DataReader("SPY", 'yahoo', minDate, yesterday).reset_index()
df_ftse = web.DataReader("VUKE.L", 'yahoo', minDate, yesterday).reset_index()
df_spy = df_spy[["Date", "Adj Close"]]
df_ftse = df_ftse[["Date", "Adj Close"]]

# Merge them together
df_markets = pd.merge(df_spy, df_ftse, on="Date", how="outer", sort=True)
df_markets = df_markets.rename(columns={"Date": "date", "Adj Close_x": "spy", "Adj Close_y": "ftse"})

df_markets = fillDays(df_markets, ["spy", "ftse"])

In [5]:
########################
#  Get exchange rates  #
########################

# Get any necessary exchange rates
df_exchange_USDGBP = web.DataReader("GBPUSD=X", 'yahoo', minDate, yesterday).reset_index()
df_exchange_USDGBP = df_exchange_USDGBP[["Date", "Close"]]
df_exchange_USDGBP = df_exchange_USDGBP.rename(columns={"Date": "date", "Close": "close_currency"})
df_exchange_USDGBP = fillDays(df_exchange_USDGBP, ["close_currency"])

In [6]:
import yahoo_fin.stock_info as si



def getAttribute_stats(df, attribute):
    result = ""

    try:
        result = df.loc[df["Attribute"] == attribute]["Value"].reset_index(drop=True)[0]
        if str(result) == "nan":
            result = "-"
    except:
        result = "-"

    return result

def getAttribute_quote(list, attribute):
    result = ""

    try:
        result = list[attribute]
        if str(result) == "nan":
            result = "-"
    except:
        result = "-"

    return result

def printInfo(t):
    print(t)
    print("Name:\t\t",      getAttribute_quote(si.get_quote_data(t),    "longName"))
    print("Sector:\t\t",    getAttribute_stats(si.get_company_info(t),  "sector"))
    print("Industry:\t",    getAttribute_stats(si.get_company_info(t),  "industry"))
    print("Price:\t\t",     getAttribute_quote(si.get_quote_data(t),    "regularMarketPrice"))
    print("Analyst:\t",     getAttribute_quote(si.get_quote_data(t),    "averageAnalystRating"))
    print("Market Cap:\t",  getAttribute_quote(si.get_quote_data(t),    "marketCap"))
    print("Trail EPS:\t",   getAttribute_quote(si.get_quote_data(t),    "epsTrailingTwelveMonths"))
    print("Fwd EPS:\t",     getAttribute_quote(si.get_quote_data(t),    "epsForward"))
    print("Trail P/E:\t",   getAttribute_quote(si.get_quote_data(t),    "trailingPE"))
    print("Fwd P/E:\t",     getAttribute_quote(si.get_quote_data(t),    "forwardPE"))
    print("Beta:\t\t",      getAttribute_stats(si.get_stats(t),         "Beta (5Y Monthly)"))
    print("52w Change:\t",  getAttribute_stats(si.get_stats(t),         "52-Week Change 3"))
    print("Fwd Dv Yld:\t",  getAttribute_stats(si.get_stats(t),         "Forward Annual Dividend Yield 4"))
    print()

print(getAttribute_quote(si.get_quote_data("AAPL"), "averageAnalystRating"))

for ticker in tickers_all:
    try:
        printInfo(ticker)
    except:
        print()



-
XOM
Name:		 Exxon Mobil Corporation
Sector:		 -
Industry:	 -
Price:		 72.17
Analyst:	 -
Market Cap:	 307493273600
Trail EPS:	 -1.386
Fwd EPS:	 5.88
Trail P/E:	 -
Fwd P/E:	 12.273809
Beta:		 1.35
52w Change:	 -
Fwd Dv Yld:	 5.54%

PLUG
Name:		 Plug Power Inc.
Sector:		 -
Industry:	 -
Price:		 21.06
Analyst:	 -
Market Cap:	 12138056704
Trail EPS:	 -1.443
Fwd EPS:	 -0.25
Trail P/E:	 -
Fwd P/E:	 -84.24
Beta:		 1.55
52w Change:	 -
Fwd Dv Yld:	 -

JPM
Name:		 JPMorgan Chase & Co.
Sector:		 -
Industry:	 -
Price:		 145.08
Analyst:	 -
Market Cap:	 439351574528
Trail EPS:	 15.809
Fwd EPS:	 11.97
Trail P/E:	 9.177052
Fwd P/E:	 12.1203
Beta:		 1.12
52w Change:	 -
Fwd Dv Yld:	 2.64%

HUKX.L
Name:		 HSBC ETFs Public Limited Company - HSBC FTSE 100 UCITS ETF

BARC.L
Name:		 Barclays PLC
Sector:		 -
Industry:	 -
Price:		 199.24
Analyst:	 -
Market Cap:	 33377681408
Trail EPS:	 31.3
Fwd EPS:	 26.21
Trail P/E:	 6.3654957
Fwd P/E:	 0.07601679
Beta:		 1.5
52w Change:	 -
Fwd Dv Yld:	 1.53%

MCD
Name:		 Mc

In [ ]:
#####################
#  Get Ticker Data  #
#####################
df_closing_all = []

for ticker in tickers_all:

    df_closing_ticker = []

    # Get the currency for the current ticker
    tickerCurrency = tickers_currency.loc[tickers_currency["ticker"] == ticker, "currency"].values[0]

    # Get the first transaction date for the ticker
    date_first = df_buys.loc[df_buys["ticker"] == ticker]["date"].min()
    
   

    # Get the last transaction date for the ticker, or the current date if it's still being held
    date_last = yesterday
    if ticker not in tickers_current:
        date_last = df_sells.loc[df_sells["ticker"] == ticker]["date"].max()

    # Get the close prices for each day
    df_closing_ticker = web.DataReader(ticker, 'yahoo', date_first, date_last).reset_index()

    # Make sure we have a 'first' date
    while date_first != df_closing_ticker["Date"].min():
        date_first = date_first - timedelta(days=1)
        df_closing_ticker = web.DataReader(ticker, 'yahoo', date_first, date_last).reset_index()

    # Format the df
    df_closing_ticker = df_closing_ticker.rename(columns={"Date": "date", "Close": "close_native"})
    df_closing_ticker = df_closing_ticker[["date", "close_native"]]
    df_closing_ticker = fillDays(df_closing_ticker, "close_native", date_first, date_last)   
    df_closing_ticker["ticker"] = ticker

    # If it's USD, divide by the exchange rate for the relevant day
    if tickerCurrency == "USD":
        df_closing_ticker = pd.merge(df_closing_ticker, df_exchange_USDGBP, how="left", on="date")
        df_closing_ticker["close_native"] = df_closing_ticker["close_native"]/df_closing_ticker["close_currency"]
        df_closing_ticker.drop(columns=["close_currency"], inplace=True)

    # If it's GBp, divide by 100 to convert to £
    elif tickerCurrency == "GBP":
        df_closing_ticker["close_native"] = df_closing_ticker["close_native"]/100
    else:
        print("ERROR: Unidentified Currency - ", ticker)

    df_closing_ticker = df_closing_ticker.rename(columns={"close_native": "close"})

    df_closing_all.append(df_closing_ticker)

# Concatenate all data
df_closing_all = pd.concat(df_closing_all)

In [ ]:
df_closing_all.to_csv('./closingPrices.csv', index=False)
df_markets.to_csv('./markets.csv', index=False)
df_tickerInfo.to_csv('./tickerInfo.csv', index=False)